In [4]:
# import libraries
import math
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from statsmodels.graphics.tsaplots import plot_acf
import arch
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance 
from sklearn.linear_model import LinearRegression

In [122]:
returns = pd.read_excel('gmo_analysis_data.xlsx',sheet_name=2,index_col='Date')
factors = pd.read_excel('gmo_analysis_data.xlsx',sheet_name=1,index_col='Date')
rf = pd.read_excel('gmo_analysis_data.xlsx',sheet_name=3,index_col='Date')
excessret = returns.subtract(rf['US3M'],axis=0).dropna()
excessret

,SPY,GMWAX
Date,,
1996-11-30,0.068729,0.040906
1996-12-31,-0.028150,-0.015630
1997-01-31,0.057494,0.010443
1997-02-28,0.005216,0.017916
1997-03-31,-0.048597,-0.019610
...,...,...
2021-06-30,0.022431,-0.011838
2021-07-31,0.024362,-0.006491
2021-08-31,0.029727,0.004641


In [475]:
def getPerformanceMetrics(data,tr,maxDD = False):

    data_desc = data.describe().loc[['mean','std','min']]

    data_desc.loc['mean'] = data_desc.loc['mean']*12 # annualize
    data_desc.loc['std'] = data_desc.loc['std']*np.sqrt(12) # annualize
    data_desc.loc['sharpe_ratio'] = data_desc.loc['mean'] / data_desc.loc['std']
    data_desc.loc['min'] = data_desc.loc['min']
    
    data_desc.loc['skewness'] = data.skew()
    data_desc.loc['excess_kurtosis'] = data.kurt()-3
    data_desc.loc['VaR05'] = np.quantile(data,.05,axis=0)
    data_desc.loc['CVaR05'] = np.mean(data<=data_desc.loc['VaR05'])
    
    def getMaxDDInfo(tr):
        cumRet = (1+tr).cumprod()
        cumMax = cumRet.cummax()
        dd = (cumRet/cumMax)-1
        # print(dd.min())
        
        maxDD = pd.DataFrame([tuple(dd.min())],
                            columns=tr.columns,
                            index=['maxDrawdown'])

        maxDD_dt = []
        # loop through each column in dd (drawdown DataFrame)
        for col,val in dd.iteritems():
            mdd = maxDD[col]['maxDrawdown']
            
            troughDate = val[val==mdd].index[0].strftime("%Y-%m-%d")
            peakDate = val[:troughDate].iloc[::-1].idxmax().strftime("%Y-%m-%d")
            recoveryDate = val[troughDate:].idxmax().strftime("%Y-%m-%d")

            maxDD_dt.append([peakDate,troughDate,recoveryDate])

        maxDD_dt = pd.DataFrame(list(zip(*maxDD_dt)), #transpose: [[1,2],[3,4]] -> [[1,3],[2,4]]
                                columns=data.columns,
                                index=['mD_peakDate','mD_troughDate','mD_recoveryDate'])
        
        return pd.concat([maxDD,maxDD_dt])
    
    if maxDD:
        return pd.concat([data_desc,getMaxDDInfo(tr)]).transpose()
    else:
        return data_desc.transpose()

## 2 Analyzing GMO

In [476]:
perf1 =getPerformanceMetrics(excessret.loc[:'2011'],returns.loc[:'2011'],maxDD = True).iloc[1]
perf2 =getPerformanceMetrics(excessret.loc['2012':],returns.loc['2012':], maxDD = True).iloc[1]
perf3 =getPerformanceMetrics(excessret, returns,maxDD = True).iloc[1]

In [477]:
summary = pd.concat({'Inception-2011':perf1, '2012-Present':perf2, 'Inception-Present':perf3},axis=1)
print('Question 1 and Question 2')
summary

Question 1 and Question 2


,Inception-2011,2012-Present,Inception-Present
mean,0.015827,0.059305,0.032928
std,0.125011,0.085303,0.11111
min,-0.149179,-0.11865,-0.149179
sharpe_ratio,0.126603,0.695229,0.29636
skewness,-1.169319,-0.942042,-1.223872
excess_kurtosis,0.086252,1.84408,1.039857
VaR05,-0.059806,-0.03065,-0.044915
CVaR05,0.054945,0.050847,0.05
maxDrawdown,-0.355219,-0.167513,-0.355219
mD_peakDate,1997-09-30,2019-12-31,1997-09-30


a) 

b)

In [388]:
def ratio_stats(returns, market_return, annualization=1):
    
    stats = pd.DataFrame(index=returns.columns)
    
    
    X = sm.add_constant(market_return) # add a const to X data
    
    for fund in stats.index:
        y = returns[fund]
        results = sm.OLS(y, X, missing='drop').fit()
        alpha = results.params[0]
        stats.loc[fund, r'$R^{2}$'] = results.rsquared
        stats.loc[fund, 'Alpha'] = alpha*annualization
        stats.loc[fund, 'Information Ratio'] = (alpha / results.resid.std()) * np.sqrt(annualization)
#         stats.loc[fund, 'Market Beta'] = beta
        for i in range(1,len(results.params)):
            stats.loc[fund, results.params.index[i]] = results.params[i]
            stats.loc[fund, str(results.tvalues.index[i])+'_t-stats'] = round(results.tvalues[i], 3)
            stats.loc[fund, str(results.pvalues.index[i])+'_p-values'] = round(results.pvalues[i], 3)
#         stats.loc[fund, 'Treynor Ratio'] = (y.mean() / beta) * annualization
        pred[fund]= results.predict(X)
    
    
    return stats

In [389]:
reg1= ratio_stats(pd.DataFrame(excessret.loc[:'2011','GMWAX']),pd.DataFrame(excessret.loc[:'2011','SPY']), annualization=12).rename(index={'GMWAX':'Inception-2011'})
reg2= ratio_stats(pd.DataFrame(excessret.loc['2012':,'GMWAX']),pd.DataFrame(excessret.loc['2012':,'SPY']), annualization=12).rename(index={'GMWAX':'2012-Present'})
reg3= ratio_stats(pd.DataFrame(excessret.loc[:,'GMWAX']),pd.DataFrame(excessret.loc[:,'SPY']), annualization=12).rename(index={'GMWAX':'Inception-Present'})

summary = pd.concat([reg1,reg2,reg3])
print('Question 3')
summary

Question 3


,$R^{2}$,Alpha,Information Ratio,SPY,SPY_t-stats,SPY_p-values
Inception-2011,0.507129,-0.005751,-0.065529,0.539615,13.609,0.0
2012-Present,0.763293,-0.028521,-0.687211,0.568297,19.341,0.0
Inception-Present,0.566804,-0.013511,-0.184759,0.546054,19.746,0.0


b) The GMWAX is low beta strategy throughout the sample.

c) No significant alpha from the strategy till 2011. After that we can observe -ve alpha for subsample from 2012-Present and Inception-Present.

## 3 Forecast Regressions

In [390]:
pred =pd.DataFrame()
lagreg1= ratio_stats(pd.DataFrame(returns['SPY']),pd.DataFrame(factors.iloc[:,0].shift(1)), annualization=12).rename(index={'SPY':'RegDP'})
pred =pred.rename(columns={'SPY':'PredRegDP'})
lagreg2= ratio_stats(pd.DataFrame(returns['SPY']),pd.DataFrame(factors.iloc[:,1].shift(1)), annualization=12).rename(index={'SPY':'RegEP'})
pred =pred.rename(columns={'SPY':'PredRegEP'})
lagreg3= ratio_stats(pd.DataFrame(returns['SPY']),pd.DataFrame(factors.shift(1)), annualization=12).rename(index={'SPY':'RegFF3'})
pred =pred.rename(columns={'SPY':'PredRegFF3'})
summary2 = pd.concat([lagreg1,lagreg2,lagreg3])
print('Question 1')
summary2.T

Question 1


,RegDP,RegEP,RegFF3
$R^{2}$,0.006830,0.007773,0.015022
Alpha,-0.070916,-0.048765,-0.113495
Information Ratio,-0.486760,-0.334878,-0.782250
DP,0.007897,NaN,0.006421
DP_t-stats,1.534000,NaN,1.220000
DP_p-values,0.126000,NaN,0.223000
EP,NaN,0.002933,0.002528
EP_t-stats,NaN,1.637000,1.379000
EP_p-values,NaN,0.103000,0.169000
US10Y,NaN,NaN,-0.001302


In [391]:
pred.head()

,PredRegDP,PredRegEP,PredRegFF3
Date,,,
1993-02-28,NaN,NaN,NaN
1993-03-31,0.016359,0.008957,0.012021
1993-04-30,0.015964,0.008869,0.011624
1993-05-31,0.016359,0.008957,0.011995
1993-06-30,0.016280,0.008781,0.011636


In [394]:
wt_pred=(pred*100).multiply(returns['SPY'],axis=0)

In [395]:
wt_pred=wt_pred.join(returns['SPY']).dropna()
wt_pred.head()

,PredRegDP,PredRegEP,PredRegFF3,SPY
Date,,,,
1993-03-31,0.036656,0.020071,0.026936,0.022408
1993-04-30,-0.040850,-0.022695,-0.029744,-0.025589
1993-05-31,0.044119,0.024157,0.032349,0.026970
1993-06-30,0.005970,0.003220,0.004267,0.003667
1993-07-31,-0.007828,-0.004164,-0.005729,-0.004855


In [479]:
perfstat =getPerformanceMetrics(wt_pred,wt_pred,maxDD = True)
regres =ratio_stats(wt_pred.iloc[:,:3],pd.DataFrame(wt_pred.iloc[:,3:4]), annualization=12)

In [480]:
perfstat.T

,PredRegDP,PredRegEP,PredRegFF3,SPY
mean,0.117959,0.119968,0.135414,0.111393
std,0.151062,0.132113,0.149929,0.14619
min,-0.211526,-0.119437,-0.170175,-0.165187
sharpe_ratio,0.780866,0.90807,0.903189,0.761975
skewness,-0.495731,0.015385,0.099945,-0.618233
excess_kurtosis,3.165963,-0.670112,1.752469,-1.747465
VaR05,-0.057902,-0.054939,-0.064473,-0.069397
CVaR05,0.052326,0.052326,0.052326,0.052326
maxDrawdown,-0.641513,-0.384958,-0.519345,-0.50798
mD_peakDate,2007-10-31,2007-10-31,2007-10-31,2007-10-31


In [481]:
regres

,$R^{2}$,Alpha,Information Ratio,SPY,SPY_t-stats,SPY_p-values
PredRegDP,0.828754,0.013172,0.210708,0.940695,40.683,0.0
PredRegEP,0.779833,0.031071,0.501224,0.798050,34.805,0.0
PredRegFF3,0.685469,0.040830,0.485577,0.849104,27.301,0.0


In [508]:
withrf = wt_pred.join(rf)
addsumm=getPerformanceMetrics(withrf.loc['2000':'2011'],withrf.loc['2000':'2011'])
addsumm

,mean,std,min,sharpe_ratio,skewness,excess_kurtosis,VaR05,CVaR05
PredRegDP,0.036507,0.182066,-0.211526,0.200516,-0.352679,3.131254,-0.073185,0.055556
PredRegEP,0.036712,0.140681,-0.119437,0.260963,0.549732,0.852477,-0.062526,0.055556
PredRegFF3,0.059687,0.160069,-0.170175,0.372884,0.223090,2.653448,-0.080942,0.055556
SPY,0.018159,0.162858,-0.165187,0.111501,-0.389757,-2.393762,-0.081272,0.055556
US3M,0.023062,0.005785,0.000008,3.986632,0.519445,-4.103977,0.000035,0.055556


In [509]:
withrf['excesspr']=withrf['PredRegFF3']-withrf['US3M']

In [510]:
negperiods =len(withrf[withrf['excesspr'] < 0])/len(withrf)
len(withrf[withrf['excesspr'] < 0])

129

In [511]:
print(str(round(negperiods*100,2))+'%')

37.5%


## 4 Out-of-Sample Forecasting

In [347]:
from statsmodels.regression.rolling import RollingOLS

In [486]:
X = sm.add_constant(factors[['EP']]).shift(1).dropna()
y = returns['SPY'].iloc[1:]
min_obv = 60
err_x, err_null, predOOS = [], [], []

for i in range(min_obv, len(y)):
    
    ### Data up to t
    currX = X.iloc[:i]
    currY = y.iloc[:i]

    ### Fit the model 
    model = sm.OLS(currY, currX, missing='drop').fit()
    
    ### Use the model to predict next SPY returns using the most recent x values
    pred = model.predict(X.iloc[[i]])[0]
    predOOS.append(pred)
    ### Forecast error of the regression
    err_x.append(y.iat[i] - pred)
    
    ### Null error is the actual value - the mean of previous values
    err_null.append(y.iat[i] - currY.mean())

### Calculate out-of-sample r2 using the errors we calculated
r_sqr_oos = 1 - np.square(err_x).sum() / np.square(err_null).sum()
print('OOS r-squared:' + str(round(r_sqr_oos, 4)))

OOS r-squared:-0.0074


In [503]:
predOOSdf = pd.DataFrame(predOOS,index=returns['SPY'].iloc[61:].index,columns=['predOOS'])
wt_predOOS = (predOOSdf*100).multiply(returns['SPY'],axis=0)
wt_predOOS= wt_predOOS.join(returns['SPY']).dropna()
wt_predOOS.head()

,predOOS,SPY
Date,,
1998-03-31,0.045644,0.048758
1998-04-30,0.014350,0.012791
1998-05-31,-0.022508,-0.020769
1998-06-30,0.035103,0.042591
1998-07-31,-0.014576,-0.013514


In [504]:
pred =pd.DataFrame()
regresOOS =ratio_stats(wt_predOOS.iloc[:,:1],pd.DataFrame(wt_predOOS.iloc[:,1:2]), annualization=12)
regresOOS

,$R^{2}$,Alpha,Information Ratio,SPY,SPY_t-stats,SPY_p-values
predOOS,0.225378,0.040106,0.279084,0.508105,9.058,0.0


In [512]:
withrfOOS = wt_predOOS.join(rf)
addsummOOS=getPerformanceMetrics(withrfOOS['2000':'2011'],withrfOOS['2000':'2011'])
addsummOOS

,mean,std,min,sharpe_ratio,skewness,excess_kurtosis,VaR05,CVaR05
predOOS,0.038768,0.195919,-0.238485,0.197877,-0.406251,1.755661,-0.085329,0.055556
SPY,0.018159,0.162858,-0.165187,0.111501,-0.389757,-2.393762,-0.081272,0.055556
US3M,0.023062,0.005785,0.000008,3.986632,0.519445,-4.103977,0.000035,0.055556


In [513]:
withrfOOS['excesspr']=withrfOOS['predOOS']-withrfOOS['US3M']
negperiodsOOS =len(withrfOOS[withrfOOS['excesspr'] < 0])/len(withrfOOS)
len(withrfOOS[withrfOOS['excesspr'] < 0])

107

In [514]:
print(str(round(negperiodsOOS*100,2))+'%')

37.68%
